# Timescale Vector Store (PostgreSQL)


## What is Timescale Vector?
**[Timescale Vector](https://www.timescale.com/ai) is PostgreSQL++ for AI applications.**

Timescale Vector enables you to efficiently store and query millions of vector embeddings in `PostgreSQL`.
- Enhances `pgvector` with faster and more accurate similarity search on millions of vectors via DiskANN inspired indexing algorithm.
- Enables fast time-based vector search via automatic time-based partitioning and indexing.
- Provides a familiar SQL interface for querying vector embeddings and relational data.

Timescale Vector scales with you from POC to production:
- Simplifies operations by enabling you to store relational metadata, vector embeddings, and time-series data in a single database.
- Benefits from rock-solid PostgreSQL foundation with enterprise-grade feature liked streaming backups and replication, high-availability and row-level security.
- Enables a worry-free experience with enterprise-grade security and compliance.

## How to use Timescale Vector
Timescale Vector is available on [Timescale](https://www.timescale.com/ai), the cloud PostgreSQL platform. (There is no self-hosted version at this time.)

**LlamaIndex users get a 90-day free trial for Timescale Vector.**
- To get started, [signup](https://console.cloud.timescale.com/signup?utm_campaign=vectorlaunch&utm_source=llamaindex&utm_medium=referral) to Timescale, create a new database and follow this notebook!
- See the [Timescale Vector explainer blog](https://www.timescale.com/blog/how-we-made-postgresql-the-best-vector-database/?utm_campaign=vectorlaunch&utm_source=llamaindex&utm_medium=referral) for details and performance benchmarks.
- See the [installation instructions](https://github.com/timescale/python-vector) for more details on using Timescale Vector in python.

## Installation

In [ ]:
!pip install -q llama-index-embeddings-openai
!pip install -q llama-index-vector-stores-timescalevector
!pip install -q llama-index

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 24.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.0/108.0 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 257.5/257.5 kB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 46.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 38.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 49.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 286.1/286.1 kB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

# Imports

In [ ]:
import openai
import os
import textwrap
import timescale_vector
from datetime import datetime, timedelta
from llama_index.core import SimpleDirectoryReader
from llama_index.core import StorageContext
from llama_index.core import VectorStoreIndex
from llama_index.core.vector_stores import VectorStoreQuery, MetadataFilters
from llama_index.vector_stores.timescalevector import TimescaleVectorStore
from timescale_vector import client
from typing import List, Tuple

In [ ]:
import llama_index.core as li
print(li.__version__)

0.10.19


### Setup OpenAI API Key & Timescale service URL
To create embeddings for documents loaded into the index, let's configure your OpenAI API key:

In [ ]:
# openai.api_key = "sk-....."
# TIMESCALE_SERVICE_URL = "postgres://tsdbadmin:lza0h2jdh74kbsak@<id>.tsdb.cloud.timescale.com:<port>/tsdb?sslmode=require"
from google.colab import userdata
TIMESCALE_SERVICE_URL = userdata.get('TIMESCALE_SERVICE_URL')
openai.api_key = userdata.get('OPENAI_KEY')

# Similarity Search with time-based filtering

Time-based vector search functionality is helpful for applications like:
- Storing and retrieving LLM response history (e.g. chatbots)
- Finding the most recent embeddings that are similar to a query vector (e.g recent news).
- Constraining similarity search to a relevant time range (e.g asking time-based questions about a knowledge base)

### Extract content and metadata from git log CSV file

In [ ]:
import pandas as pd

df = pd.read_csv("commit_history.csv")
df.dropna(inplace=True)
df = df.astype(str)
df = df[:1000]

In [ ]:
df.head(3)

,commit,author,date,change summary,change details
0,44e41c12ab25e36c202f58e068ced262eadc8d16,Lakshmi Narayanan Sreethar<lakshmi@timescale.com>,Tue Sep 5 21:03:21 2023 +0530,Fix segfault in set_integer_now_func,When an invalid function oid is passed to set_...
1,e66a40038e3c84fb1a68da67ad71caf75c64a027,Bharathy<satish.8483@gmail.com>,Sat Sep 2 09:24:31 2023 +0530,Fix server crash on UPDATE of compressed chunk,UPDATE query with system attributes in WHERE c...
2,c6a930897e9f9e9878db031cc7fb6ea79d721a74,Jan Nidzwetzki<jan@timescale.com>,Tue Aug 29 21:13:51 2023 +0200,Use Debian Bookworm for 32-bit tests,"So far, we have used Debian Buster (10) for ou..."


In [ ]:
def create_uuid(date_string: str):
    if date_string is None:
        return None
    time_format = "%a %b %d %H:%M:%S %Y %z"
    datetime_obj = datetime.strptime(date_string, time_format)
    uuid = client.uuid_from_time(datetime_obj)
    return str(uuid)

def split_name(input_string: str) -> Tuple[str, str]:
    if input_string is None:
        return None, None
    start = input_string.find("<")
    end = input_string.find(">")
    name = input_string[:start].strip()
    return name

In [ ]:
def create_date(input_string: str) -> datetime:
    if input_string is None:
        return None
    # Define a dictionary to map month abbreviations to their numerical equivalents
    month_dict = {
        "Jan": "01",
        "Feb": "02",
        "Mar": "03",
        "Apr": "04",
        "May": "05",
        "Jun": "06",
        "Jul": "07",
        "Aug": "08",
        "Sep": "09",
        "Oct": "10",
        "Nov": "11",
        "Dec": "12",
    }

    # Split the input string into its components
    components = input_string.split()
    # Extract relevant information
    day = components[2]
    month = month_dict[components[1]]
    year = components[4]
    time = components[3]
    timezone_offset_minutes = int(
        components[5]
    )  # Convert the offset to minutes
    timezone_hours = timezone_offset_minutes // 60  # Calculate the hours
    timezone_minutes = (
        timezone_offset_minutes % 60
    )  # Calculate the remaining minutes
    # Create a formatted string for the timestamptz in PostgreSQL format
    timestamp_tz_str = (
        f"{year}-{month}-{day} {time}+{timezone_hours:02}{timezone_minutes:02}"
    )
    return timestamp_tz_str

Next, we'll define a function to create a `TextNode` for each git log entry. We'll use the helper function `create_uuid()` we defined above to create a uuid for each node based on its timestampe. And we'll use the helper functions `create_date()` and `split_name()` above to extract relevant metadata from the git log entry and add them to the node.

In [ ]:
from llama_index.core.schema import TextNode, NodeRelationship, RelatedNodeInfo

# Create a Node object from a single row of data
def create_node(row):
    record = row.to_dict()
    record_name = split_name(record["author"])
    record_content = (
        str(record["date"])
        + " "
        + record_name
        + " "
        + str(record["change summary"])
        + " "
        + str(record["change details"])
    )
    # Can change to TextNode as needed
    node = TextNode(
        id_=create_uuid(record["date"]),
        text=record_content,
        metadata={
            "commit": record["commit"],
            "author": record_name,
            "date": create_date(record["date"]),
        },
    )
    return node

In [ ]:
nodes = [create_node(row) for _, row in df.iterrows()]

In [ ]:
nodes[0]

TextNode(id_='8b407680-4c01-11ee-ad79-03d90024f5fc', embedding=None, metadata={'commit': '44e41c12ab25e36c202f58e068ced262eadc8d16', 'author': 'Lakshmi Narayanan Sreethar', 'date': '2023-09-5 21:03:21+0850'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='Tue Sep 5 21:03:21 2023 +0530 Lakshmi Narayanan Sreethar Fix segfault in set_integer_now_func When an invalid function oid is passed to set_integer_now_func, it finds out that the function oid is invalid but before throwing the error, it calls ReleaseSysCache on an invalid tuple causing a segfault. Fixed that by removing the invalid call to ReleaseSysCache.  Fixes #6037 ', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n')

Next we'll create vector embeddings of the content of each node so that we can perform similarity search on the text associated with each node. We'll use the `OpenAIEmbedding` model to create the embeddings.

In [ ]:
# Create embeddings for nodes
from llama_index.embeddings.openai import OpenAIEmbedding

embedding_model = OpenAIEmbedding()

for node in nodes:
    node_embedding = embedding_model.get_text_embedding(
        node.get_content(metadata_mode="all")
    )
    node.embedding = node_embedding

Let's examine the first node in our collection to see what it looks like.

In [ ]:
print(nodes[0].get_content(metadata_mode="all"))

commit: 44e41c12ab25e36c202f58e068ced262eadc8d16
author: Lakshmi Narayanan Sreethar
date: 2023-09-5 21:03:21+0850

Tue Sep 5 21:03:21 2023 +0530 Lakshmi Narayanan Sreethar Fix segfault in set_integer_now_func When an invalid function oid is passed to set_integer_now_func, it finds out that the function oid is invalid but before throwing the error, it calls ReleaseSysCache on an invalid tuple causing a segfault. Fixed that by removing the invalid call to ReleaseSysCache.  Fixes #6037


In [ ]:
print(nodes[0].get_embedding())

[-0.005558566655963659, 0.00213569775223732, 0.006601247470825911, -0.02713846229016781, -0.03931984677910805, 0.026074208319187164, -0.02012014202773571, -0.01236835028976202, -0.0418798103928566, -0.0049077901057899, 0.05689441040158272, 0.0069320290349423885, 0.014453711919486523, 0.009161208756268024, 0.0008444814593531191, -0.012979577295482159, -0.002288504270836711, -0.016639744862914085, 0.005418344400823116, -0.0019469365943223238, 0.008851999416947365, 0.01187936868518591, -0.024751082062721252, -0.002849394688382745, 0.0031675919890403748, -0.00027213068096898496, -0.0033815214410424232, -0.039434902369976044, 0.0015217744512483478, -0.01215981412678957, 0.017243782058358192, -0.0230252668261528, -0.017517035827040672, -0.023658065125346184, -0.016150765120983124, -0.025441408157348633, -0.01626581884920597, -0.006105075124651194, -0.000942906946875155, 0.007176519371569157, -0.0015073927352204919, -0.009865916334092617, -0.013037104159593582, -0.024650409817695618, -0.00230

In [ ]:
len(nodes[0].get_embedding())

1536

### Load documents and metadata into TimescaleVector vectorstore
Now that we have prepared our nodes and added embeddings to them, let's add them into our TimescaleVector vectorstore.

We'll create a Timescale Vector instance from the list of nodes we created.

First, we'll define a collection name, which will be the name of our table in the PostgreSQL database.

We'll also define a time delta, which we pass to the `time_partition_interval` argument, which will be used to as the interval for partitioning the data by time. Each partition will consist of data for the specified length of time. We'll use 7 days for simplicity, but you can pick whatever value make sense for your use case -- for example if you query recent vectors frequently you might want to use a smaller time delta like 1 day, or if you query vectors over a decade long time period then you might want to use a larger time delta like 6 months or 1 year.

Then we'll add the nodes to the Timescale Vector vectorstore.

In [ ]:
# Create a timescale vector store and add the newly created nodes to it
ts_vector_store = TimescaleVectorStore.from_params(
    service_url=TIMESCALE_SERVICE_URL,
    table_name="li_commit_history",
    time_partition_interval=timedelta(days=7),
)
_ = ts_vector_store.add(nodes)

### Querying vectors by time and similarity

Now that we have loaded our documents into TimescaleVector, we can query them by time and similarity.

TimescaleVector provides multiple methods for querying vectors by doing similarity search with time-based filtering Let's take a look at each method below.

First we define a query string and get the vector embedding for the query string.

In [ ]:
# Define query and generate embedding for it
query_str = "What's new with TimescaleDB functions?"
embed_model = OpenAIEmbedding()
query_embedding = embed_model.get_query_embedding(query_str)

Then we set some variables which we'll use in our time filters.

In [ ]:
# Time filter variables for query
start_dt = datetime(
    2023, 8, 1, 22, 10, 35
)  # Start date = 1 August 2023, 22:10:35
end_dt = datetime(
    2023, 8, 30, 22, 10, 35
)  # End date = 30 August 2023, 22:10:35
td = timedelta(days=7)  # Time delta = 7 days

Method 1: Filter within a provided start date and end date.

In [ ]:
# Query the vector database
vector_store_query = VectorStoreQuery(
    query_embedding=query_embedding, similarity_top_k=5
)

# return most similar vectors to query between start date and end date date range
# returns a VectorStoreQueryResult object
query_result = ts_vector_store.query(
    vector_store_query, start_date=start_dt, end_date=end_dt
)
query_result

VectorStoreQueryResult(nodes=[TextNode(id_='22747180-31f1-11ee-b8b6-3da627e2841c', embedding=None, metadata={'commit': ' 7aeed663b9c0f337b530fd6cad47704a51a9b2ec', 'author': 'Dmitry Simonenko', 'date': '2023-08-3 14:30:23+0500'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='Thu Aug 3 14:30:23 2023 +0300 Dmitry Simonenko Feature flags for TimescaleDB features This PR adds several GUCs which allow to enable/disable major timescaledb features:  - enable_hypertable_create - enable_hypertable_compression - enable_cagg_create - enable_policy_create', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'), TextNode(id_='22747180-31f1-11ee-97fb-e8b8c6e661f5', embedding=None, metadata={'commit': ' 7aeed663b9c0f337b530fd6cad47704a51a9b2ec', 'author': 'Dmitry Simonenko', 'date': '2023-08-3 14:30:23+0500'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys

Let's inspect the nodes that were returned from the similarity search:

In [ ]:
# for each node in the query result, print the node metadata date
for node in query_result.nodes:
    print("-" * 80)
    print(node.metadata["date"])
    print(node.get_content(metadata_mode="all"))

--------------------------------------------------------------------------------
2023-08-3 14:30:23+0500
commit:  7aeed663b9c0f337b530fd6cad47704a51a9b2ec
author: Dmitry Simonenko
date: 2023-08-3 14:30:23+0500

Thu Aug 3 14:30:23 2023 +0300 Dmitry Simonenko Feature flags for TimescaleDB features This PR adds several GUCs which allow to enable/disable major timescaledb features:  - enable_hypertable_create - enable_hypertable_compression - enable_cagg_create - enable_policy_create
--------------------------------------------------------------------------------
2023-08-3 14:30:23+0500
commit:  7aeed663b9c0f337b530fd6cad47704a51a9b2ec
author: Dmitry Simonenko
date: 2023-08-3 14:30:23+0500

Thu Aug 3 14:30:23 2023 +0300 Dmitry Simonenko Feature flags for TimescaleDB features This PR adds several GUCs which allow to enable/disable major timescaledb features:  - enable_hypertable_create - enable_hypertable_compression - enable_cagg_create - enable_policy_create
------------------------------

## RAG Application

In [ ]:
index = VectorStoreIndex.from_vector_store(ts_vector_store)
query_engine = index.as_query_engine(vector_store_kwargs = ({"start_date": start_dt,
                                                             "end_date":end_dt}))

query_str = "What's new with TimescaleDB functions? When were these changes made and by whom?"
response = query_engine.query(query_str)
print(str(response))

The new feature added to TimescaleDB functions is the implementation of several GUCs that enable or disable major TimescaleDB features. These changes were made on August 3, 2023, by Dmitry Simonenko.


In [ ]:
from pprint import pprint

In [ ]:
pprint(str(response))

('The new feature added to TimescaleDB functions is the implementation of '
 'several GUCs that enable or disable major TimescaleDB features. These '
 'changes were made on August 3, 2023, by Dmitry Simonenko.')
